<h1 align=center><font size = 6>Best Neighborhood to Rent in Hamilton</font></h1>

## Introduction

Hamilton is a Canadian port city on the western tip of Lake Ontario, it is famous for its amazing escarpment views and numerous waterfalls, it is a great place to live in. Many people working in Toronto are moving to Hamilton because of the great natural views, cheaper rent and convenient transportation. The objective of this project is to analyze the neighborhoods in Hamilton based on the featured venues and the average rent price to help people who are considering moving to Hamilton to find the best place to rent that suit their needs. 

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Download and Explore Dataset</a>

2. <a href="#item2">Explore Neighborhoods in Hamilton, ON</a>

3. <a href="#item3">Analyze Each Neighborhood</a>

4. <a href="#item4">Cluster Neighborhoods</a>

5. <a href="#item5">Examine Clusters</a>    
</font>
</div>

First, download the libraries needed for the project.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


## 1. Download and Explore Dataset

In [3]:
rent_data = pd.read_csv('rent.csv')
rend_data

FileNotFoundError: [Errno 2] File rent.csv does not exist: 'rent.csv'

## 2. Explore Neighborhoods in Hamilton, ON

## 3. Analyze Each Neighborhood

## 4. Cluster Neighborhoods

## 5. Examine Clusters